## About the Smart Village python microservice
We are writing a Microservice that we can run on openshift. The goal is to read Map data from the official Smart Village api and push map data to your own Smart Village instance that you have deployed using the previous notebooks. We will do this by writing a Python program and a Dockerfile that will containerize the service.


#### Setting values we will use for this program
For testing this application, we want to talk to the environment you have just deployed on the previous notebook. 
The above commands will first set environment variables for `MAP_SERVER_URL`, `CLIENT_ID` and `CLIENT_SECRET` and use them to create a openshift secret we can later download to update our workbench in the next section.

In [ ]:
%%bash
export MAP_SERVER_URL="https://$(oc get route/smartvillage-web -o jsonpath={.spec.host})"
export CLIENT_ID="$(oc get secret/smartvillage -o jsonpath={.data.CLIENT_ID} | base64 -d)"
export CLIENT_SECRET="$(oc get secret/smartvillage -o jsonpath={.data.CLIENT_SECRET} | base64 -d)"
oc create secret generic python-microservice --from-literal=MAP_SERVER_URL=$MAP_SERVER_URL --from-literal=CLIENT_ID=$CLIENT_ID --from-literal=CLIENT_SECRET=$CLIENT_SECRET
echo DONE

#### Now we want to make sure we update our `OpenShift AI workbench` with this newly created python-microservice secret.

1. Let's download the `python-microservice secret` we created at the beginning that has the Environment Variables we need to pass to our workbench.
#### Deploying on Openshift

2. From your [developer sandbox](https://console.redhat.com/openshift/sandbox) open the openshift view.

<img src="pictures/microservice/0007.png" />

3. Select the <img src="pictures/microservice/0023.png" /> view on the top left corner.

4. Under `Workloads` select `Secrets`.

<img src="pictures/microservice/0024.png" />

5. On the secrets page search for `python-microservice` and click on the secret.

<img src="pictures/microservice/0025.png" />

6. On the secret details page, switch to the <img src="pictures/microservice/0026.png" /> view.  

7. On the `YAML` view, select <img src="pictures/microservice/0027.png" /> on the bottom right of the page. This will download the secret as a YAML file on your system which we will use in the next steps.

8. Lets go back into your workbench from the introduction of the course.

9. In a new tab open your OpenShift Sandbox, click on the apps button
<img src="pictures/100002010000003000000024AAAC041571052865.png" />
at the top,

then click
<img src="pictures/100002010000012600000028AA3F546B7EAF857C.png" />
to log into OpenShift AI.

Click the button to
<img src="pictures/10000201000000A000000020B028AB197DEBE3A3.png" />.

Log into OpenShift AI by clicking on the
<img src="pictures/100002010000006C0000001DABF5B58FF6B1D253.png" />
button.

Once you are in OpenShift AI, click on the menu button
<img src="pictures/100002010000002E000000216426608B65255A13.png" />,
then click
<img src="pictures/10000201000000B200000027F58611BB363F3154.png" />.

Then click on the name of your data science project
<img src="pictures/100002010000008F00000034E5BB257B3DE6397F.png" />,
for my Red Hat user it’s computate-dev.

10. You should see your `Workbenches` and the workbench you created listed.

<img src="pictures/microservice/0001.png" />

11. On the right hand side of your workbench there is 3 dots, click on them and select `Edit Workbench`
##CHANGE workbench picture for python name
<img src="pictures/microservice/0002.png" />

12. Select `Environment Variables` then `Add more variables`

<img src="pictures/microservice/0003.png" />

13. Select `Secret` from the Environment variable type dropdown menu. Then select `Upload`.

14. Click on Upload and select the python-microservice.yaml we downloaded on step 1.

<img src="pictures/microservice/0004.png" />

15. Now click on the <img src="pictures/microservice/0005.png" /> button.

To make sure that we have properly updated our workbench we can issue the following commands. You should see the 3 environment variables printed before DONE.

In [ ]:
%%bash
echo "My MAP_SERVER_URL is: $MAP_SERVER_URL"
echo "My CLIENT_ID is: $CLIENT_ID"
echo "My CLIENT_SECRET is: $CLIENT_SECRET"
echo DONE

We also want the cient_id and client_secret. We want read those from the secret object and set it to a environment variable we are going to use shortly.

## Python Microservice.
This program will have a http endpoint that will trigger the updates to the map data. To expose an http endpoint we will be using [Flask](https://flask.palletsprojects.com) as a dependency. In order to handle http requests we will use [requests](https://pypi.org/project/requests/). When dealing with objects that can change value at runtime a common way to change the behavior of the application is using environment variables, for this we will use the [os](https://docs.python.org/3/library/os.html) module that is part of the Python library. Finally we want to read data from the Smart Village api based on our current timezone, for that we we will use [pytz](https://pypi.org/project/pytz/) to handle the timezone and use the types datetime & timezone from the [datetime](https://docs.python.org/3/library/datetime.html) module that is also part of the Python library.

Let's install modules needed that are not part of the python standard library.

In [ ]:
%%bash
pip install requests flask pytz
echo DONE

#### Setting imports and app
Above we are going to import all the needed modules and setup our Flask app.

In [ ]:
import os
import pytz
import requests
from flask import Flask
from datetime import datetime, timezone
# Define the Flask app.
app = Flask(__name__)

#### Using environment variables.
The above section of code will read all 3 environment variables and setup the static value of the Smart Village api. THe last `if` statement is checking if the the credentials were provided. You can also add a check for the MAP_SERVER_URL.

In [ ]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
map_server = os.getenv('MAP_SERVER_URL')
token_auth_url = "https://sso.smartabyarsmartvillage.org/auth/realms/SMARTVILLAGE/protocol/openid-connect/token"

if client_id is None or client_secret is None:
    print("No CLIENT_ID or CLIENT_SECRET environment variable or defined...Exiting")
    exit(1)

#### Getting the Access token
To communicate with the API we need to have permissions, in this case we are using an access_token as authentication mechanism. To get the access token that we can re-use to make api calls, we need to make an initial request to the authentication url. This is when we will use the client_id, client_secret to do a request to the server. Above we are using the requests module to do a post. The data or payload we send to the server is inside `token_auth_payload`.

In [ ]:
token_auth_payload =  {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}
response = requests.post(token_auth_url, token_auth_payload)

#### Error handling
When talking to the authentication server we will get a valid token if the status code is 200, if the status code is not 200 we will not have a way to talk to the apis to get map data. To be able to debug what is going wrong we should print the status code and the error. The code above saves the access_token if successful and prints the status_code. If you encounter issues you can print additional information from the response object. What is supported can be seen on the requests [documentation](https://requests.readthedocs.io/en/latest/user/quickstart/#response-content).

In [ ]:
access_token = ""
if response.status_code == 200:
    access_token = response.json().get('access_token')
    print(f"Token: {access_token}")
else:
    print(f"Status: {response.status_code}")

#### GET api values
Let's setup a couple of variables for communicating with the APIs. The `api_uri` is the URL of the official Smart Village deployment that we will use to get the map data from. The `map_data_api` is the api to do a GET request from that official Smart Village deployment.

In [ ]:
api_uri = "https://www.smartabyarsmartvillage.org"
map_data_api = "/api/map-result"

#### GET function `get_map_data()`
This section of the code should be in it's own function at the end. We should name the function `get_map_data` and will have `no arguments` and at the end of the function we will return `map_data`. For now we are writing it directly in the script.

* First, we are getting the current time in our timezone.

In [ ]:
# Get current time in America/New_York timezone
utc_time = datetime.utcnow()
my_timezone_name = 'America/New_York'
my_timezone = pytz.timezone(my_timezone_name)
current_my_time = utc_time.replace(tzinfo=timezone.utc).astimezone(my_timezone)
formatted_time = current_my_time.strftime("%Y-%m-%dT%H:%M:%S.000") + ("[%s]" % my_timezone_name)
print(formatted_time)

* Second, we construct the parameters we need to pass to the map results api.

In [ ]:
headers = {
 'Authorization': f'Bearer {access_token}',
}
params = {"rows": "100","fq": f"dateTime:{formatted_time}"}


* Third, we make the request passing our access_token and parameters.

In [ ]:
map_result = requests.get(url=api_uri+map_data_api, params=params, headers=headers)

* Fourth, we get the data points if successful. However in the function we would want to return map_data.

In [ ]:
if map_result.status_code == 200:
    map_data = map_result.json()
    print(f"GET Request Successful. Data: {map_data}")
else:
    print(f"GET Status code: {map_result.status_code}")
print(f"My map data is: {map_data}")
#return map_data

#### Declare the POST api endpoint.
`smart_traffic_light_api` is the api endpoint we need tot talk to on your Smart Village deployment. We will concatenate your Smart Village URL (MAP_SERVER_URL) with this api endpoint to do a PUT request and update the map values from the data we got from the official server in the previous section.

In [ ]:
smart_traffic_light_api= "/api/smart-traffic-light-import"

#### PUT function  `update_data(map_data)`
This section should also be in function but we are writing it initially as part of the script. This function we should call `update_data` and take the agument `(map_data)`. This function will not return anything at the end just print out debug statements.

* First, we setup the headers for the PUT request

In [ ]:
headers = {
'Authorization': f'Bearer {access_token}',
'Content-Type': 'application/json',
}

* Second, we use a for loop to go thru the data we got from the Official Smart Village server and save only the coordinates.

In [ ]:
all_coordinates = []
for item in map_data['list']:
    if 'location' in item and 'coordinates' in item['location']:
        coordinates = item['location']['coordinates']
        all_coordinates.append(coordinates)

* Third, we prepare the JSON payload for the PUT we call it `light_data`

In [ ]:
light_data = {
    "pk": "veberod-intersection-1",
    "saves": ["entityId", "smartTrafficLightName", "location", "areaServed"],
    "entityId": "urn:ngsi-ld:SmartTrafficLight:SmartTrafficLight-veberod-intersection-1",
    "smartTrafficLightName": "Veberöd intersection 1",
    "areaServed": [
    ]
}

* Fourth, we modify light_data to include all the coordinates.

In [ ]:
light_data['areaServed'] = [{"type": "Point", "coordinates": coordinates} for coordinates in all_coordinates[1:]]

* Finally, we make the request updating your Smart Village deployment with the coordinate data.
We can print as much as debug information as we need. In this care we have 3 print statements. In the function these can be combined or modified as you see fit.

In [ ]:
if map_server is not None:
    put_response = requests.put(url=map_server+smart_traffic_light_api, json={"list":[light_data]}, headers=headers)
    if put_response.status_code == 200:
        put_data = put_response.json()
        print(f"Successful PUT, Updated: {put_data}")
    else:
        print(f"PUT Failed: {put_response.status_code}, {put_response.text}")
else:
    print("No MAP_SERVER_URL environment variable defined, printing payload to terminal:")
    print(f"My light_data is: {light_data}")

#### Setting up the service code
The `@app.route('/health')` annotation on top of a function is what tells flask what function to call when we access that http endpoint.
The `check_data` function will call the `get_map_data` function and then update your Smart Village deployment with that data by calling `update_data`. 
These are the functions you must create from the above sections.

In [ ]:
@app.route('/health')
def check_data():
    map_data = get_map_data()
    update_data(map_data)
    return "App is running"

To open the port used for talking to the service we need to define in code or at runtime. 
For this instance we are going to programmatically setup the host/port that way we can easily import this application to OpenShift using source to image.

In [ ]:
if __name__ == '__main__':
     app.run(host='0.0.0.0', port=8080, debug=False)

## Building a container with my application.
In this section we will create a container image that holds our application and lets us run it locally.

First let's create a folder to hold our code and the files we need for our application.
These files are:
- `app.py` is the python script we built in the previous sections.
- `Dockerfile` this is commonly called a Dockerfile or Containerfile, this holds the instructions on how to build the container.
- `.github/workflows/build.yml` a automation script (github workflow) that builds and push the container to a registry.
- `.gitignore` This file tells git not to track certain files, for us it is ignoring the course notebook checkpoints, which are not relevant to the application.

In [ ]:
%%bash
mkdir  ~/smartvillage-service
touch ~/smartvillage-service/Dockerfile
touch ~/smartvillage-service/app.py
mkdir -p ~/smartvillage-service/.github/workflows/
touch ~/smartvillage-service/.github/workflows/build.yml
echo "**/.ipynb_checkpoints" >> ~/smartvillage-service/.gitignore
echo DONE

#### Creating a Containerfile/Dockerfile

The First line in this Dockerfile we create on `touch ~/smartvillage-service/Dockerfile` should be our base image.
The in our case the base image will have most of the dependencies we need to run our application.
By using `registry.access.redhat.com/ubi9/python-311` we get python and pip pre-installed on a stable and reliable Linux container based on RHEL content.

```
FROM registry.access.redhat.com/ubi9/python-311
```

On next line we are copying the python application into the container.
```
COPY app.py .
```

Now since this environment does not have all the dependencies we need, we must run pip like we did at the beginning of our notebook, but this time in the container build:
```
RUN pip install --no-cache-dir Flask requests pytz
```

Our application uses 3 environment variables, we need to define them so we can set them at runtime in the container file, the next lines do that for us.
```
ENV CLIENT_ID=
ENV CLIENT_SECRET=
ENV MAP_SERVER_URL=
```

We want our application to accept connections on port `8080` to accomplish that we need to EXPOSE that port:
```
EXPOSE 8080
```

Finally the CMD instruction, tells the container what command to run when it's started. In our case we want it to run python with our script as the argument.
```
CMD ["python", "app.py"]
```

#### Building our container using automation.
The file we created in the step above `.github/workflows/build.yml` will control how we are building the container image with podman in github.

The first part of the file defines the name of the workflow and how it is triggered. In this case it will be triggered every time you push to the main branch.

```
name: Build
on:
  push:
    branches: ["main"]
```

Like our container the workflow uses environment variables to configure the build, we are defining the registry, which will be the github one for this example, the name of the image, the location and your user name and password. The great thing is that all this is automatically read from your github account, the only thing that that you can optionally change is the IMAGE_NAME.
```
env:
  REGISTRY: ghcr.io
  IMAGE_NAME: "python-microservice"
  IMAGE_REGISTRY: ghcr.io/${{ github.repository_owner }}
  REGISTRY_USER: ${{ github.actor }}
  REGISTRY_PASSWORD: ${{ github.token }}
```

Finally we got a job with 3 steps. Notice that each step must have the same indentation as the first one. 

First this defines a `podman-build` job that uses the default ubuntu-latest container from github to run podman and build your Dockerfile. The first step just checkout the repository you have created with your python application and Dockerfile, this uses [the checkout action](https://github.com/actions/checkout).
```
jobs:
  podman-build:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout repo
        uses: actions/checkout@v3

The second step builds the Dockerfile using the [buildah-build action](https://github.com/redhat-actions/buildah-build) from Red Hat. 
```
      - name: Build container image
        uses: redhat-actions/buildah-build@v2
        with:
          context: "."
          containerfiles: Dockerfile
          image: ${{ env.IMAGE_NAME }}
          layers: false
          oci: true
```

Finally on the last step we need to push that image built on the second step, to do that we will use the [push-to-registry action](https://github.com/redhat-actions/push-to-registry). This action uses the environment variables we setup before to get the credentials automatically and push to the ghcr.io registry. Here we are also defining the tag that we are going to push to, in this case `latest`.
```
      - name: Push to ghcr.io
        uses: redhat-actions/push-to-registry@v2.6
        with:
          image: ${{ env.IMAGE_NAME }}
          tags: "latest"
          registry: ${{ env.IMAGE_REGISTRY }}
          username: ${{ env.REGISTRY_USER }}
          password: ${{ env.REGISTRY_PASSWORD }}
```



#### Update the app.py with the python application we built.
You have tested the application on the workbench, now we need to update `~/smartvillage-service/app.py` with the code with the proper functions and get it ready with the other files.

At this point you should have all 4 files:

- ~/smartvillage-service/Dockerfile
- ~/smartvillage-service/app.py
- ~/smartvillage-service/.github/workflows/build.yml
- ~/smartvillage-service/.gitignore

#### Creating your github repository
We want to create a github repository following this [link.](https://docs.github.com/en/repositories/creating-and-managing-repositories/quickstart-for-repositories#create-a-repository) We just want to do the first step which is to create the repository, do not commit any code yet.

Once you have your repository, we want to create a personal access token to be able push our code to github.

Click [here to generate a GitHub personal access token](https://github.com/settings/tokens?type=beta). Make sure you are creating a fine-grained access token which can grant "read and write" access for "Content" and "Workflows" to the your smartvillage-service repo.

Give your token a note that you can remember.

You will want to remember the random generated token value for later. You can copy it to the clipboard with the button on the right.

To finish your git setup we want to setup your email, name and token/credentials by replacing the information below with your information and issuing the commands:

```
git config --global user.email "MyEmail@example.com"
git config --global user.name "FirstName LastName"
git config --global credential.helper store
```

#### Allow github workflows to push the container image
To allow the automatic github action to push the built image to the container registry, we need to navigate to the setting page of your new repository.
```
https://github.com/<YOUR-USER-NAME>/smartvillage-service/settings/actions
```
<img src="pictures/microservice/0006.png" />

In the actions page under `Workflow permissions` near the end, select `Read and write permissions` and click `save`.

#### Committing your code to github
 
First let's initialize our repository. For this we want to create a README.md file. md is the extension of Markdown, which is a common way to write documentation.

```
cd ~/smartvillage-service/
git init
echo "My smartvillage service" >> README.md
git add README.md
git commit -m "first commit"
git branch -M main
```

Now for the next steps you will need to make sure to use your git repository address. 

```
git remote add origin https://github.com/<YOUR-USER-NAME>/smartvillage-service.git
git push -u origin main
```

Now you can confirm that the README.md exists on your repository by navigating to github.com/<YOUR-USER-NAME>/smartvillage-service.

Once you have confirmed the Readme is in place, we can commit our code and push it to github.

```
cd ~/smartvillage-service/
git add Dockerfile
git add app.py
git add .github
git add .gitignore
```

You can confirm all the correct files are added to staging by issuing:

```
git status
```

if it looks good we want to commit:

```
git commit -m "Add service, Dockerfile and github workflow."
```

Finally we push again to our main branch:

```
git push -u origin main
```

Finally you should see your code updated on your repository by navigating to github.com/<YOUR-USER-NAME>/smartvillage-service and your workflow building the container should be running at: github.com/<YOUR-USER-NAME>/smartvillage-service/actions

If everything goes well, we should have your container built under:
```
https://github.com/<YOUR-USER-NAME>/smartvillage-service/pkgs/container/python-microservice
```

which should show you the address to use the container, for example
```
ghcr.io/<YOUR-USER-NAME>/python-microservice:latest
```

We will use this address on the next step.

#### Deploying on Openshift

Openshift allow you to quickly deploy an application by using a container image. The code we pushed to our github repository and built with the action can now easily be used in Openshift thanks to the container image we now have. Whe system will start the container, and make a Kubernetes deployment, setup all the routes and services needed to talk to the service.

1. From your [developer sandbox](https://console.redhat.com/openshift/sandbox) open the openshift view.

<img src="pictures/microservice/0007.png" />

2. In the Developer view, select the <img src="pictures/microservice/0008.png" /> button, and click on the <img src="pictures/microservice/0009.png" /> tile.

3. On the Deploy image page, paste a link to your `ghcr.io` container from the last step under `Image name from external registry`.

<img src="pictures/microservice/0010.png" />

4. Under General-> Application: Select <img src="pictures/microservice/0011.png" />.

5. Under Name give it a name like: `python-microservice`.

6. Resource type: `Deployment`

<img src="pictures/microservice/0012.png" />

7. Leave everything else as is, but click on the `Health checks`.

<img src="pictures/microservice/0013.png" />

8. Click on the <img src="pictures/microservice/0014.png" /> button.

9. For `type` select: HTTP GET.

10. set Path to: `/health`.  - This is what we defined as the application path that triggers the data update on our application.

11. Change period to `3`. - This will hit the /health api from our application every 3 seconds and update our maps.

12. Press the <img src="pictures/microservice/0015.png" /> button.


#### Passing Environment Variable to your deployment.
Now we have our application deployment ready. The last step is to update it to pass the 3 environment variables from the beginning of this notebook to the container.

1. Select the <img src="pictures/microservice/0016.png" /> view from the top left corner, you might need to change it from Developer to Administrator.

2. Select `Workloads` then `Deployments`.

<img src="pictures/microservice/0017.png" />

3. Click on the 3 dots <img src="pictures/microservice/0018.png" /> on the right side of your deployment.

4. Select <img src="pictures/microservice/0019.png" from the dropdown.

5. Close to the bottom of the page you should see `Environment Variables` select <img src="pictures/microservice/0020.png" />.

6. On name put `CLIENT_ID`, from the `select resource` dropdown select `python-microservice` which is the secret we created at the beginning, and from the `select key` dropdown select `CLIENT_ID`.

<img src="pictures/microservice/0021.png" />

7. Repeat for `CLIENT_SECRET` and `MAP_SERVER_URL` environment variables and press the <img src="pictures/microservice/0022.png" /> button.

This will trigger a deployment update and your application should be ready. If you monitor your smart village deployment you should see the data being updated every 3 seconds or so.

#### Access you smart village updated map.

Now we want to access the map on your deployment of the Smart Village.
After running the above code you will get an URL that you can copy and paste on your browser to see your map and should be updated every few seconds with new data.

In [ ]:
%%bash
echo "https://$(oc get route/smartvillage-web -o jsonpath={.spec.host})/smart-traffic-light/1"
echo DONE